<a href="https://colab.research.google.com/github/waristh/Indonesia_Covid19_With_Plotly/blob/main/Plotly_DataCovidIndonesia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install plotly==4.13.0

     |████████████████████████████████| 13.1 MB 44 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
import plotly.express as px
import numpy as np

In [ ]:
# #Agar Mapnya di view di browser
# import plotly.io as pio
# pio.renderers.default = 'browser'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/GIS

/content/gdrive/MyDrive/GIS


In [ ]:
#Buat function untuk ambil data covid indonesia
import requests
def get_json(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return json.loads(response.content.decode('utf-8'))
    else:
        return None

In [ ]:
#Grep data hari ini
import datetime
import pandas as pd
import json
record_date = datetime.date.today().strftime('%Y-%m-%d')
Propinsi = ['ACEH','SUMATERA_UTARA','SUMATERA_BARAT','SUMATERA_SELATAN','RIAU','JAMBI','BENGKULU','KEPULAUAN_BANGKA_BELITUNG','KEPULAUAN_RIAU','LAMPUNG',
            'DKI_JAKARTA','BANTEN','JAWA_BARAT','JAWA_TENGAH','JAWA_TIMUR','DAERAH_ISTIMEWA_YOGYAKARTA',
            'BALI','NUSA_TENGGARA_BARAT','NUSA_TENGGARA_TIMUR',
            'KALIMANTAN_BARAT','KALIMANTAN_TIMUR','KALIMANTAN_TENGAH','KALIMANTAN_SELATAN','KALIMANTAN_UTARA',
            'SULAWESI_BARAT','SULAWESI_TENGGARA','SULAWESI_SELATAN','SULAWESI_TENGAH','GORONTALO','SULAWESI_UTARA',
            'MALUKU','MALUKU_UTARA','PAPUA','PAPUA_BARAT',]

In [ ]:
try:
  Data_Covid = pd.DataFrame(columns=['tanggal', 'KASUS', 'MENINGGAL', 'SEMBUH', 'DIRAWAT_OR_ISOLASI',
        'AKUMULASI_KASUS', 'AKUMULASI_SEMBUH', 'AKUMULASI_MENINGGAL',
        'AKUMULASI_DIRAWAT_OR_ISOLASI','Propinsi'])

  for prov in Propinsi:
    data_prov = pd.io.json.json_normalize(get_json(f'https://data.covid19.go.id/public/api/prov_detail_{prov}.json')['list_perkembangan'][0:])
    #menambahkan kolom provinsi berdasarkan provinsinya
    data_prov['Propinsi'] = f'{prov}'
    #meng convert kolom tanggal dari format Timestamp ke Tanggal
    data_prov['tanggal'] = pd.DataFrame(pd.to_datetime(data_prov['tanggal'].values.astype(dtype='datetime64[ms]')))

    Data_Covid = pd.concat([Data_Covid,data_prov])
    #save data per provinsi ke excel
    data_prov.to_excel(f'DataCovid_{prov}.xlsx',index=False)
except:
  pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


In [ ]:
import glob
#Membaca semua data kemudian menggabungkannya menjadi satu
Data_Covid = pd.DataFrame(columns=['tanggal', 'KASUS', 'MENINGGAL', 'SEMBUH', 'DIRAWAT_OR_ISOLASI',
       'AKUMULASI_KASUS', 'AKUMULASI_SEMBUH', 'AKUMULASI_MENINGGAL',
       'AKUMULASI_DIRAWAT_OR_ISOLASI','Propinsi'])
files = glob.glob(r'DataCovid*.xlsx')
for f in files:
    print(f)
    Data = pd.read_excel(f'{f}')
    Data_Covid = pd.concat([Data,Data_Covid])
    print(Data_Covid.shape)



In [ ]:
Data_Covid

In [ ]:
#Merubah data angka meenjadi integer
Data_Covid[['KASUS', 'MENINGGAL', 'SEMBUH', 'DIRAWAT_OR_ISOLASI',
       'AKUMULASI_KASUS', 'AKUMULASI_SEMBUH', 'AKUMULASI_MENINGGAL',
       'AKUMULASI_DIRAWAT_OR_ISOLASI',]] = Data_Covid[['KASUS', 'MENINGGAL', 'SEMBUH', 'DIRAWAT_OR_ISOLASI',
       'AKUMULASI_KASUS', 'AKUMULASI_SEMBUH', 'AKUMULASI_MENINGGAL',
       'AKUMULASI_DIRAWAT_OR_ISOLASI',]].astype(int)

In [ ]:
Data_Covid.info()

In [ ]:
# !pip install geopandas

In [ ]:
# import geopandas
# indo_geojson = geopandas.read_file('indonesia-prov.geojson')
# print(indo_geojson)
# print(indo_geojson.columns)

In [ ]:
# Data_Covid = Data_Covid.merge(indo_geojson, on='Propinsi')
# Data_Covid.tail()

# Set Data Geo JSON

In [ ]:
!pip install geojson

In [ ]:
import geojson
with open('indonesia-prov.geojson') as f:
    gj = geojson.load(f)
indo_geojson = gj
indo_geojson

In [ ]:
indo_geojson['features']

In [ ]:
# for x in range(0,33):
#     print(f'\'{indo_geojson["features"][x]["properties"]["Propinsi"]}\' : \'{indo_geojson["features"][x]["properties"]["kode"]}\',')
    

In [ ]:
id_dict = {'NUSA_TENGGARA_BARAT' : '52',
'GORONTALO' : '75',
'SULAWESI_TENGGARA' : '74',
'DAERAH_ISTIMEWA_YOGYAKARTA' : '34',
'JAWA_TENGAH' : '33',
'BANTEN' : '36',
'JAWA_TIMUR' : '35',
'MALUKU_UTARA' : '82',
'MALUKU' : '81',
'KALIMANTAN_SELATAN' : '63',
'KALIMANTAN_BARAT' : '61',
'SULAWESI_SELATAN' : '73',
'DKI_JAKARTA' : '31',
'JAWA_BARAT' : '32',
'PAPUA' : '91',
'NUSA_TENGGARA_TIMUR' : '53',
'BALI' : '51',
'RIAU' : '14',
'KEPULAUAN_RIAU' : '21',
'SULAWESI_BARAT' : '76',
'SULAWESI_TENGAH' : '72',
'KALIMANTAN_TIMUR' : '64',
'SULAWESI_UTARA' : '71',
'PAPUA_BARAT' : '92',
'SUMATERA_UTARA' : '12',
'KEPULAUAN_BANGKA_BELITUNG' : '19',
'SUMATERA_BARAT' : '13',
'KALIMANTAN_TENGAH' : '62',
'SUMATERA_SELATAN' : '16',
'JAMBI' : '15',
'LAMPUNG' : '18',
'BENGKULU' : '17',
'ACEH' : '11',}

Data_Covid['id'] = Data_Covid['Propinsi'].map(id_dict)
Data_Covid.tail()


# Filter hanya data tahun 2021

In [ ]:
#Format tanggalnya harus di rubah jadi string
Data_Covid['tanggal'] = Data_Covid['tanggal'].astype(str)

Data_Covid_2021 = Data_Covid.loc[(Data_Covid['tanggal'] > '2020-12-31')]
Data_Covid_2021.head()

# Data Covid sesuai setting tanggal

In [ ]:
Data_Covid_Tgl = Data_Covid.loc[(Data_Covid['tanggal'] > '2021-07-1') & (Data_Covid['tanggal'] < '2021-08-30')]
#Format tanggalnya harus di rubah jadi string
Data_Covid_Tgl['tanggal'] = Data_Covid_Tgl['tanggal'].astype(str)
Data_Covid_Tgl.head()

# View Indonesia dari World Map berdasarkan ID nya

In [ ]:
fig=px.choropleth(Data_Covid,
                  geojson=indo_geojson,  
                  locations='id',        #column in dataframe
                  hover_name='Propinsi',  
                  projection='orthographic',
                  height=700
                  )
# fig.update_geos(fitbounds="locations", visible=False)
fig.show()

# View Indonesia dari Asia Map

In [ ]:
fig=px.choropleth(Data_Covid,
                  geojson=indo_geojson,
                  # featureidkey='properties.Propinsi', 
                  locations='id',        #column in dataframe
                  hover_name='Propinsi',
                  scope='asia',
                  height=700
                  )
fig.show()

# View only Indonesian Map

In [ ]:
fig=px.choropleth(Data_Covid,
                  geojson=indo_geojson, 
                  locations='id',        #column in dataframe,
                  hover_name='Propinsi',
                  height=700,
                  )
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [ ]:
fig=px.choropleth(Data_Covid_Tgl,
                  geojson=indo_geojson,
                  # featureidkey='properties.Propinsi',   
                  locations='id',        #column in dataframe
                  animation_frame='tanggal',       #dataframe
                  color='AKUMULASI_KASUS',  #dataframe
                  # color_continuous_scale='Inferno',
                  color_continuous_scale=px.colors.diverging.Tealrose,
                  # color_continuous_scale=px.colors.qualitative.Light24,
                  # color_continuous_midpoint=2,
                  # projection='orthographic',
                  hover_name='Propinsi',
                  hover_data=["AKUMULASI_KASUS","AKUMULASI_MENINGGAL","AKUMULASI_SEMBUH"],
                  title='PERKEMBANGAN KASUS COVID DI INDONESIA PER PROVINSI' ,  
                  height=700
                  )
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

In [ ]:
# Data_Covid_Tgl['LOG_AKUMULASI_KASUS'] = np.log10(Data_Covid_Tgl['AKUMULASI_KASUS'])

# Melihat timeseries perkembangan kasus

## Membuat Pivot Data Baru dari Kasus per hari

In [ ]:
Data_Kasus =  Data_Covid[['Propinsi','tanggal','KASUS']]
Data_Kasus = Data_Kasus.pivot(index='tanggal', columns='Propinsi')['KASUS']
Data_Kasus = Data_Kasus.dropna()
Data_Kasus = Data_Kasus.astype(int)

Data_Kasus

In [ ]:
#buat kolom tanggal dan merubahnya menjadi string
Data_Kasus['tanggal'] = Data_Kasus.index
Data_Kasus = Data_Kasus.reset_index(drop=True)
# Data_Kasus['tanggal'] = Data_Kasus['tanggal'].astype(str)
Data_Kasus

## View Graph

In [ ]:
fig = px.line(Data_Kasus, x="tanggal", y=Data_Kasus.columns,
              hover_data={"tanggal": "|%d %b, %Y"},
              height=1000,
              title='Timeseries Grafik Perkembangan Kasus Per Hari')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
fig.show()

## View Graph berdasarkan range tanggal yg kita mau view

In [ ]:
fig = px.line(Data_Kasus, x="tanggal", y=Data_Kasus.columns,
              hover_data={"tanggal": "|%d %b, %Y"},
              range_x=['2021-01-01','2021-07-30'],
              height=1000,
              title='Timeseries Grafik Perkembangan Kasus Per Hari')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
fig.show()

## View Graph Jika ingin dengan slider

In [ ]:
fig = px.line(Data_Kasus, x="tanggal", y=Data_Kasus.columns,
              hover_data={"tanggal": "|%d %b, %Y"},
              height=1000,
              title='Timeseries Grafik Perkembangan Kasus Per Hari')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")

fig.update_xaxes(rangeslider_visible=True)
fig.show()

## View Graph Jika ingin dengan slider & range selector button

In [ ]:
fig = px.line(Data_Kasus, x="tanggal", y=Data_Kasus.columns,
              hover_data={"tanggal": "|%d %b, %Y"},
              height=800,
              color_discrete_sequence=px.colors.qualitative.Light24,
              title='Timeseries Grafik Perkembangan Kasus Per Hari')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="1w", step="day", stepmode="backward"), 
            dict(count=14, label="2w", step="day", stepmode="backward"),                       
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

# Membuat Dataset baru untuk Data yg sembuh dan meninggal

In [ ]:
Data_MENINGGAL = Data_Covid[['Propinsi','tanggal','MENINGGAL']]
Data_MENINGGAL = Data_MENINGGAL.pivot(index='tanggal', columns='Propinsi')['MENINGGAL']
Data_MENINGGAL = Data_MENINGGAL.dropna()
Data_MENINGGAL = Data_MENINGGAL.astype(int)

#buat kolom tanggal dan merubahnya menjadi string
Data_MENINGGAL['tanggal'] = Data_MENINGGAL.index
Data_MENINGGAL = Data_MENINGGAL.reset_index(drop=True)
Data_MENINGGAL['tanggal'] = Data_MENINGGAL['tanggal'].astype(str)

Data_MENINGGAL

In [ ]:
Data_SEMBUH = Data_Covid[['Propinsi','tanggal','SEMBUH']]
Data_SEMBUH = Data_SEMBUH.pivot(index='tanggal', columns='Propinsi')['SEMBUH']
Data_SEMBUH = Data_SEMBUH.dropna()
Data_SEMBUH = Data_SEMBUH.astype(int)

#buat kolom tanggal dan merubahnya menjadi string
Data_SEMBUH['tanggal'] = Data_SEMBUH.index
Data_SEMBUH = Data_SEMBUH.reset_index(drop=True)
Data_SEMBUH['tanggal'] = Data_SEMBUH['tanggal'].astype(str)

Data_SEMBUH

# View Graph Data Sembuh & Meninggal

In [ ]:
fig = px.line(Data_MENINGGAL, x="tanggal", y=Data_MENINGGAL.columns,
              hover_data={"tanggal": "|%d %b, %Y"},
              height=800,
              title='Timeseries Grafik Meninggal Per Hari')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="1w", step="day", stepmode="backward"), 
            dict(count=14, label="2w", step="day", stepmode="backward"),                      
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

In [ ]:
fig = px.line(Data_SEMBUH, x="tanggal", y=Data_SEMBUH.columns,
              hover_data={"tanggal": "|%d %b, %Y"},
              height=800,
              title='Timeseries Grafik SEMBUH Per Hari')
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="1w", step="day", stepmode="backward"), 
            dict(count=14, label="2w", step="day", stepmode="backward"),                      
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

# Analisa Kasus Positif berdasarkan persentase Usia

In [ ]:
Data_Umur_Kasus = pd.DataFrame(columns=['key','doc_count','usia.value'])

for prov in Propinsi:
  Data_Umur_Kasus_prov = pd.io.json.json_normalize(get_json(f'https://data.covid19.go.id/public/api/prov_detail_{prov}.json')['data']['kasus']['kelompok_umur']['list_data'])
  #menambahkan kolom provinsi berdasarkan provinsinya
  Data_Umur_Kasus_prov['Propinsi'] = f'{prov}'
  #meng convert kolom tanggal dari format Timestamp ke Tanggal
  Data_Umur_Kasus_prov['doc_count'] = round(Data_Umur_Kasus_prov['doc_count'],2)

  Data_Umur_Kasus = pd.concat([Data_Umur_Kasus,Data_Umur_Kasus_prov])

#save data per provinsi ke excel
Data_Umur_Kasus.rename(columns={'key':'umur','doc_count':'persentase_umur_kasus'},inplace=True)
Data_Umur_Kasus.drop(columns=['usia.value'],inplace=True)
Data_Umur_Kasus.to_excel(f'Data_Umur_Kasus.xlsx',index=False)
Data_Umur_Kasus

In [ ]:
Data_Umur_Kasus = pd.read_excel('Data_Umur_Kasus.xlsx')
Data_Umur_Kasus

In [ ]:
Data_Umur_Kasus.info()

In [ ]:
import plotly.express as px

fig = px.bar(Data_Umur_Kasus, x='Propinsi', 
             y='persentase_umur_kasus',
             color="umur",
             title="PERSENTASE KASUS POSITIF BERDASARKAN USIA",
             height=700)
#set bar nya agar saling bersebelahan bukan menumpuk
fig.update_layout(barmode='group')

fig.show()



In [ ]:
pd.io.json.json_normalize(get_json(f'https://data.covid19.go.id/public/api/prov_detail_DKI_JAKARTA.json')['data']['meninggal'])

# Analisa Kasus Meninggal berdasarkan usia

In [ ]:
# pd.io.json.json_normalize(get_json(f'https://data.covid19.go.id/public/api/prov_detail_DKI_JAKARTA.json')['data']['meninggal']['kelompok_umur']['list_data'])

In [ ]:
Data_Umur_Meninggal = pd.DataFrame(columns=['key','doc_count','usia.value'])

for prov in Propinsi:
  Data_Umur_Meninggal_prov = pd.io.json.json_normalize(get_json(f'https://data.covid19.go.id/public/api/prov_detail_{prov}.json')['data']['meninggal']['kelompok_umur']['list_data'])
  #menambahkan kolom provinsi berdasarkan provinsinya
  Data_Umur_Meninggal_prov['Propinsi'] = f'{prov}'
  #meng convert kolom tanggal dari format Timestamp ke Tanggal
  Data_Umur_Meninggal_prov['doc_count'] = round(Data_Umur_Meninggal_prov['doc_count'],2)

  Data_Umur_Meninggal = pd.concat([Data_Umur_Meninggal,Data_Umur_Meninggal_prov])

#save data per provinsi ke excel
Data_Umur_Meninggal.rename(columns={'key':'umur','doc_count':'persentase_umur_meninggal'},inplace=True)
Data_Umur_Meninggal.drop(columns=['usia.value'],inplace=True)
Data_Umur_Meninggal.to_excel(f'Data_Umur_Meninggal.xlsx',index=False)
Data_Umur_Meninggal

In [ ]:
Data_Umur_Meninggal = pd.read_excel('Data_Umur_Meninggal.xlsx')
Data_Umur_Meninggal

In [ ]:
import plotly.express as px

fig = px.bar(Data_Umur_Meninggal, x='Propinsi', 
             y='persentase_umur_meninggal',
             color="umur",
             title="PERSENTASE MENINGGAL BERDASARKAN USIA",
             height=700)
fig.show()



In [ ]:
import plotly.express as px

fig = px.bar(Data_Umur_Meninggal, x='Propinsi', 
             y='persentase_umur_meninggal',
             color="umur",
             title="PERSENTASE MENINGGAL BERDASARKAN USIA",
             height=700)
#set bar nya agar saling bersebelahan bukan menumpuk
fig.update_layout(barmode='group')

fig.show()



# Analisa Data Meninggal Beserta Komorbid

In [ ]:
# pd.io.json.json_normalize(get_json(f'https://data.covid19.go.id/public/api/prov_detail_MALUKU.json')['data']['meninggal']['kondisi_penyerta']['list_data'])

In [ ]:
Data_Komorbid_Meninggal = pd.DataFrame(columns=['key','doc_count'])

for prov in Propinsi:
  try:
    Data_Komorbid_Meninggal_prov = pd.io.json.json_normalize(get_json(f'https://data.covid19.go.id/public/api/prov_detail_{prov}.json')['data']['meninggal']['kondisi_penyerta']['list_data'])
    #menambahkan kolom provinsi berdasarkan provinsinya
    Data_Komorbid_Meninggal_prov['Propinsi'] = f'{prov}'
    #meng convert kolom tanggal dari format Timestamp ke Tanggal
    Data_Komorbid_Meninggal_prov['doc_count'] = round(Data_Komorbid_Meninggal_prov['doc_count'],2)

    Data_Komorbid_Meninggal = pd.concat([Data_Komorbid_Meninggal,Data_Komorbid_Meninggal_prov])
  except:
    pass
#save data per provinsi ke excel
Data_Komorbid_Meninggal.rename(columns={'key':'komorbid','doc_count':'persentase_komorbid_meninggal'},inplace=True)
Data_Komorbid_Meninggal.to_excel(f'Data_Komorbid_Meninggal.xlsx',index=False)
Data_Komorbid_Meninggal

In [ ]:
Data_Komorbid_Meninggal = pd.read_excel('Data_Komorbid_Meninggal.xlsx')
Data_Komorbid_Meninggal

In [ ]:
import plotly.express as px

fig = px.bar(Data_Komorbid_Meninggal, x='Propinsi', 
             y='persentase_komorbid_meninggal',
             color="komorbid",
             title="PERSENTASE MENINGGAL BERDASARKAN KOMORBID",
             color_discrete_sequence=px.colors.qualitative.Light24,
             height=700)
fig.show()



In [ ]:
# pd.io.json.json_normalize(get_json(f'https://data.covid19.go.id/public/api/prov_detail_DKI_JAKARTA.json')['data']['sembuh']['kelompok_umur']['list_data'])

In [ ]:
# pd.io.json.json_normalize(get_json(f'https://data.covid19.go.id/public/api/prov_detail_DKI_JAKARTA.json')['data']['sembuh']['kondisi_penyerta']['list_data'])

# Membuat Graph data Total kasus se Indonesia

In [ ]:
Data_Total = pd.pivot_table(Data_Covid,index=["tanggal"],values=["KASUS",'MENINGGAL','SEMBUH'],aggfunc=np.sum)
Data_Total

In [ ]:
Data_Total['tanggal'] = Data_Total.index
Data_Total = Data_Total.reset_index(drop=True)
px.bar(Data_Total,x=Data_Total.tanggal,y=['KASUS','MENINGGAL','SEMBUH'])

In [ ]:
fig = px.bar(Data_Total,
             x=Data_Total.tanggal,
             y=['KASUS','MENINGGAL','SEMBUH'],
             title="TOTAL KASUS POSITIF, MMENINGGAL & SEMBUH COVID-19 DI INDONESIA",
             color_discrete_sequence=px.colors.qualitative.Light24,
             height=600)

#set bar nya agar saling bersebelahan bukan menumpuk
fig.update_layout(barmode='group')

#set tulisan x axis tanggalnya agar formatnya bulan-tahun
fig.update_xaxes(
    dtick="M1",
    tickformat="%d-%b\n%Y",
    ticklabelmode="period")

#set tombol
fig.update_xaxes(
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="1w", step="day", stepmode="backward"), 
            dict(count=14, label="2w", step="day", stepmode="backward"),                      
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
#menampilkan grafik
fig.show()

# Membuat Racing Bar plot dari akumulasi kasus per hari

In [ ]:
!pip install raceplotly

In [ ]:
import pandas as pd
from raceplotly.plots import barplot

data = Data_Covid.loc[(Data_Covid['tanggal'] > '2020-05-31')]

my_raceplot = barplot(data.sort_values(by='tanggal'),  item_column='Propinsi', value_column='AKUMULASI_KASUS', time_column='tanggal')
my_raceplot.plot(item_label = 'TOP 10 PROPINSI', value_label = 'RACING KASUS POSITIF', frame_duration = 100)

In [ ]:
my_raceplot = barplot(data.sort_values(by='tanggal'),  item_column='Propinsi', value_column='AKUMULASI_KASUS', time_column='tanggal')
my_raceplot.plot(item_label = 'TOP 10 PROPINSI', value_label = 'RACING KASUS POSITIF', frame_duration = 100,orientation='vertical')